Andrey_Bondarchuk_stepik_id_346032105

collab

https://drive.google.com/file/d/1pEkACTIDJTGpW0wJM2fOxi3NGTW7toKu/view?usp=sharing

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
# !unzip archive.zip

In [ ]:
collab = False

In [ ]:
import os
if collab and (not 'training.1600000.processed.noemoticon.csv' in os.listdir('./')):
    from google.colab import drive
    drive.mount('/content/gdrive/')
    !unzip -q /content/gdrive/My\ Drive/archive.zip

Импортируем нужные библиотеки:

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, \
                   names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cpu'

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


In [ ]:
# посмотрим
print(filtered_line)

['north', 'devon', 'coast', 'next', 'weeks', 'will', 'down', 'devon', 'again', 'sometime', 'hope', 'though']


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

### Небольшая предобработка текста

In [ ]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))
print(stopWords)

{"mustn't", 'my', 'yours', 'not', 'until', 'where', 'don', "hadn't", 'from', 'all', 'these', 'ourselves', 'wasn', 'who', 'theirs', 'in', 'to', "hasn't", "wouldn't", 'yourselves', 'we', 'some', "needn't", 'during', 'of', 'are', 'll', 'nor', "you'd", 'own', 'won', 'shouldn', 'does', 'their', 'few', 'ain', 'same', 'whom', 've', 'before', 'on', "that'll", "aren't", "isn't", 'when', 'didn', 'himself', 'isn', 'so', 'ma', 'wouldn', 'm', 'such', 'between', "you'll", 'it', 'an', 'aren', 'or', 'having', 'against', 'under', "couldn't", 'here', 'other', "you've", 'hasn', 'off', 'this', 'if', 'once', "wasn't", 'through', 'up', 'and', 'too', 'doing', 'any', 'hadn', 'very', 'needn', 'yourself', 'shan', 'themselves', 'is', 'mustn', "haven't", 'be', 'haven', 'but', 'had', 'again', 'can', 'while', "mightn't", 'was', 'for', 'your', 'now', 'has', "don't", 'y', 'more', 'by', 'than', "shouldn't", 'being', 'the', "won't", 'which', 'herself', 's', 'each', 't', 'both', 'hers', 'because', 'mightn', 'only', 'did

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def prepocess_df(strin):
    try:
        strin = strin.lower()
        strin = re.sub(r"http?://[^,\s]+,?","",strin)
        strin = re.sub(r'@[\w]+',"",strin)
        
        tokens = tokenizer.tokenize(strin.lower())
          
        filtered_line = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        filtered_line = [word for word in filtered_line if word not in stopWords]
        filtered_line = " ".join(filtered_line)
        return filtered_line
    except:
        return ""

data["tokens"] = data["text"].apply(prepocess_df)

In [ ]:
data[['emotion','text','tokens']].head(10)

,emotion,text,tokens
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda david carr third
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might result sch...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest bounds
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving
5,0,@Kwesidei not the whole crew,whole crew
6,0,Need a hug,need
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,long time rains fine thanks
8,0,@Tatiana_K nope they didn't have it,nope
9,0,@twittera que me muera ?,muera


In [ ]:
%%time
word2vec = api.load("word2vec-google-news-300")

Wall time: 1min 8s


In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


In [ ]:
# средний эмбеддинг
_emb = np.mean(emb_line, axis=0)
_emb.shape

(300,)

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"http?://[^,\s]+,?","",text)
        text = re.sub(r'@[\w]+',"",text)
        line = tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        filtered_line = [word for word in filtered_line if word not in stopWords]
        return filtered_line

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их

        # не нормализованные эмбеддинги
        #embeddings = [self.word2vec.get_vector(w) for w in tokens if w in self.word2vec]
        
        # нормализованные эмбеддинги 
        embeddings = [((self.word2vec.get_vector(w) - self.mean)/self.std) for w in tokens if w in self.word2vec] 
  
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        #embeddings = np.mean(embeddings, axis=0)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion",word2vec)

In [ ]:
# посмотрим что получилось
next(iter(dev))

{'feature': array([[-0.36991876, -1.1144205 ,  0.33723447, ...,  0.35792458,
          2.792679  , -2.6243513 ],
        [-2.044868  ,  0.15058553, -0.67700225, ...,  0.05009809,
         -1.0761887 ,  0.66978776],
        [ 0.22370882,  0.888091  , -2.4832325 , ...,  0.18854395,
          0.8179045 ,  0.31222534],
        ...,
        [ 0.9292662 ,  1.2742614 , -0.07795607, ..., -0.979371  ,
          0.74874896, -1.8027185 ],
        [ 0.13376525,  1.1946386 ,  1.9293033 , ..., -0.5168706 ,
          0.3837614 , -0.7129138 ],
        [ 0.73338914,  0.16750537, -0.20423055, ..., -0.19839445,
          0.95621556, -0.53127974]], dtype=float32),
 'target': 0}

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes],   # было [np.sum(dev[i]["feature"], axis=0) for i in example_indexes]
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
# посмотрим
pd_examples = pd.DataFrame(examples)
pd_examples.head(5)

,features,targets
0,"[0.79884803, 0.29639465, -0.22475016, 0.827617...",0
1,"[0.23503505, 0.33106375, 0.0489919, 0.18425526...",0
2,"[-0.22001284, 1.2040938, -0.2605174, 0.578202,...",0
3,"[0.21771257, -0.3414166, 0.64904827, 0.4894894...",1
4,"[0.46089336, 0.37187043, -0.133843, 0.6015578,...",0


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
#examples["transformed_features"] = # Обучи PCA на эмбеддингах слов

In [ ]:
list_features = pd_examples['features'].to_list()
X = np.array(list_features)

In [ ]:
X.shape

(1280, 300)

In [ ]:
new_X = pca.fit_transform(X)

In [ ]:
new_X.shape

(1280, 2)

In [ ]:
examples["transformed_features"] = new_X

In [ ]:
#!pip3 install bokeh

In [ ]:
#!pip3 install typing_extensions

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1115', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 0 # 4 - для collab, 0 - local

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    # было 
    #return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}
    # стало
    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

# для collab num_workers = 4
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, 
                          collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, 
                          collate_fn=average_emb)


In [ ]:
# смотрим
next(iter(train_loader))['features'].shape

torch.Size([1024, 300])

Определим функции для тренировки и теста модели:

In [ ]:
from tqdm.notebook import tqdm

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели -----
        optimizer.zero_grad()
        outp = model(features)
        preds = outp.argmax(-1)
        # --------------------------------  

        loss = criterion(outp, targets) # Посчитай лосс

        # Обнови параметры модели --------
        loss.backward()
        optimizer.step()
        # --------------------------------  

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
        
def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            outp = model(features)
            preds = outp.argmax(-1)
            
            loss = criterion(outp, targets) # Посчитай лосс
            
            acc = (preds == targets).sum(dim=0) # Посчитай точность модели
            if not len(preds)==0: 
                acc = acc / len(preds)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {float(loss):.4}, Test Acc: {float(acc):.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn
from torch.optim import Adam

from torchsummary import summary

# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 5

Создадим простой классификатор из трех полносвязных слоев.

In [ ]:
# Создадим модель

class MyClassifier(nn.Module):   
    def __init__(self, num_classes=2, vector_size =300):
        super().__init__()       
        self.classifier = nn.Sequential(
           nn.Flatten(),
           nn.Linear(vector_size, vector_size//2, bias=False), # 300 -> 150
           nn.BatchNorm1d(vector_size//2),
           nn.ELU(),
           nn.Linear(vector_size//2, vector_size//2//2, bias=False), # 150 -> 75
           nn.BatchNorm1d(vector_size//2//2),
           nn.ELU(),
           nn.Linear(vector_size//2//2, num_classes, bias=True) # 75 -> 2
         )
    def forward(self, x):
        return self.classifier(x)

In [ ]:
model = MyClassifier(num_classes, vector_size) 

In [ ]:
if torch.cuda.is_available():
    summary(model.cuda(), (1,vector_size), batch_size=1024)
else:
    summary(model, (1, vector_size), batch_size=1024)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                [1024, 300]               0
            Linear-2                [1024, 150]          45,000
       BatchNorm1d-3                [1024, 150]             300
               ELU-4                [1024, 150]               0
            Linear-5                 [1024, 75]          11,250
       BatchNorm1d-6                 [1024, 75]             150
               ELU-7                 [1024, 75]               0
            Linear-8                  [1024, 2]             152
Total params: 56,852
Trainable params: 56,852
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.17
Forward/backward pass size (MB): 7.63
Params size (MB): 0.22
Estimated Total Size (MB): 9.02
----------------------------------------------------------------


In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters()) # Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [ ]:
if torch.cuda.is_available():
    model = model.cuda()
else:
    pass 

In [ ]:
%%time

num_epochs = 5

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5217936517000198, 'Test Acc': 0.73741796875}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5089089448451996, 'Test Acc': 0.7474609375}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5065802465677262, 'Test Acc': 0.747859375}
Wall time: 16min 24s


In [ ]:
# результаты будем сохраниять в список
result_list = []

In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

res = testing(model, criterion, test_loader, device=device)
res['model'] = 'word2vec'
result_list.append(res)

print(res)

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5083348428288968, 'Test Acc': 0.747678714057508, 'model': 'word2vec'}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

### Пункт 1.  Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста

#### Разработка алгоритма по сложению эмбеддингов контекстных слов для неизвестного слова

In [ ]:
# разбиваем произвольный твит на токены

text = dev_data['text'][0]
line = tokenizer.tokenize(text.lower())
tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]

In [ ]:
# добавим искусственно неизвестное слово
tokens.insert(3,'limpopo')

In [ ]:
# смотрим
print(tokens)

['north', 'devon', 'coast', 'limpopo', 'next', 'weeks', 'will', 'down', 'devon', 'again', 'sometime', 'hope', 'though']


In [ ]:
# рассчитываем -> ембединг('limpopo') = ембединг('coast') + ембединг('next')

# окно для просмотра контекста в данном примере
context_window_size = 1

embeddings = []
iter_ = enumerate(tokens)
for idx, word in iter_:      
    if word not in word2vec:  
        #continue
        start_idx = max(0, idx - context_window_size)
        end_idx = min(len(tokens)-1, idx + context_window_size)
        emb_context_words =  word2vec.get_vector('python')*0 #нулевой ембеддинг
        for idx_context in range(start_idx, end_idx+1):
            if idx_context==idx:
                continue
            if tokens[idx_context] not in word2vec:  
                continue
            emb_context_words += word2vec.get_vector(tokens[idx_context])
        embeddings.append(emb_context_words)
    else:
        embeddings.append(word2vec.get_vector(word))

In [ ]:
emb_context_words == (word2vec.get_vector('coast') + word2vec.get_vector('next'))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

Все работает.

 Сформируем новый класс для загрузчика, с учетом нового алгоритма для ембеддингов неизвестных слов. (Как среднее из контекста, слова, входящие в контекст заданного слова, будем определять просматривая текст окном определенной ширины)

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset_context_window(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, 
                 context_window_size: int):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        
        # размер окна для просмотра контекста неизвестных слов
        self.context_window_size = context_window_size
        
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}
        

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"http?://[^,\s]+,?","",text)
        text = re.sub(r'@[\w]+',"",text)
        line = tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        filtered_line = [word for word in filtered_line if word not in stopWords]
        return filtered_line

#     def get_tokens_(self, text):
#         # Получим все токены из текста и профильтруй их
#         line = tokenizer.tokenize(text.lower())
#         filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
#         return filtered_line

    def get_embeddings_(self, tokens):
      
        embeddings = []
  
        iter_ = enumerate(tokens)
        for idx, word in iter_:      
            if  word not in word2vec: 
                start_idx = max(0, idx - self.context_window_size)
                end_idx = min(len(tokens)-1, idx + self.context_window_size)
                emb_sum_of_context_words = word2vec.get_vector('python')*0 #нулевой ембеддинг
                for idx_context in range(start_idx, end_idx+1):
                    if idx_context==idx:
                        continue
                    if tokens[idx_context] not in self.word2vec:
                        continue
                    emb = (self.word2vec.get_vector(tokens[idx_context]) - self.mean)/self.std
                    emb_sum_of_context_words += emb
                embeddings.append(emb_sum_of_context_words)
            else:
                emb = (self.word2vec.get_vector(word) - self.mean)/self.std
                embeddings.append(emb) 
        
        # было ранее
        #embeddings = [((self.word2vec.get_vector(w) - self.mean)/self.std) for w in tokens if w in self.word2vec] 
  
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
 
        return embeddings

    def __len__(self):
        return self.data.shape[0]

Определим ширину окна для определения суммового ембеддинга для неизвестного слова в два токена слева и справа.

In [ ]:
context_window_size = 2

In [ ]:
dev_context_window = TwitterDataset_context_window(dev_data, "text", "emotion", word2vec, context_window_size)

In [ ]:
next(iter(dev_context_window))

{'feature': array([[-0.36991876, -1.1144205 ,  0.33723447, ...,  0.35792458,
          2.792679  , -2.6243513 ],
        [-2.044868  ,  0.15058553, -0.67700225, ...,  0.05009809,
         -1.0761887 ,  0.66978776],
        [ 0.22370882,  0.888091  , -2.4832325 , ...,  0.18854395,
          0.8179045 ,  0.31222534],
        ...,
        [ 0.9292662 ,  1.2742614 , -0.07795607, ..., -0.979371  ,
          0.74874896, -1.8027185 ],
        [ 0.13376525,  1.1946386 ,  1.9293033 , ..., -0.5168706 ,
          0.3837614 , -0.7129138 ],
        [ 0.73338914,  0.16750537, -0.20423055, ..., -0.19839445,
          0.95621556, -0.53127974]], dtype=float32),
 'target': 0}

In [ ]:
batch_size = 1024
num_workers = 0 # 4 - для collab, 0 - local

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]
    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev_context_window) * 0.8)

train, valid = random_split(dev_context_window, [train_size, len(dev_context_window) - train_size])

#для local num_workers = 0
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, \
                          drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, \
                          drop_last=False, collate_fn=average_emb)


In [ ]:
model = MyClassifier(num_classes, vector_size) 
criterion = torch.nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters()) # Твой оптимайзер

In [ ]:
lr = 1e-2

In [ ]:
%%time

num_epochs = 5

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_context_window.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5266181662082672, 'Test Acc': 0.73361328125}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5108939216136933, 'Test Acc': 0.745859375}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5123246805667877, 'Test Acc': 0.7450390625}
Wall time: 17min 49s


In [ ]:
test_loader = DataLoader(
    # новый класс для загрузки данных --------------------------------------------------------
    TwitterDataset_context_window(test_data, "text", "emotion", word2vec, context_window_size), 
    # ----------------------------------------------------------------------------------------
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_context_window.pt", map_location=device))

res = testing(model, criterion, test_loader, device=device)
res['model'] = 'word2vec context window'
result_list.append(res)

print(res)

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.511253299518896, 'Test Acc': 0.7454697484025559, 'model': 'word2vec context window'}


### Пункт 2.  Для каждого слова текста получим его эмбеддинг из Tfidf с помощью TfidfVectorizer из sklearn. Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного

полезные примеры

tfidf gensim
https://habr.com/ru/post/542048/

TF-IDF in scikit-learn and Gensim
https://weiliu2k.github.io/CITS4012/gensim/tf-idf.html#id1

#### Соберем из наших данных корпус слов для обучения tfidf.

In [ ]:
#!pip3ainstall -U scikit-learna

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus = data['tokens'].values

In [ ]:
corpus[0]

'awww bummer shoulda david carr third'

In [ ]:
%%time
# обучаем Tfidf
vectorizer  = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(corpus)
X.shape

Wall time: 16.6 s


(1600000, 272542)

In [ ]:
X.T.shape

(272542, 1600000)

In [ ]:
# размер словаря
feature_list = vectorizer.get_feature_names_out()
len(feature_list)

272542

#### Понизим размерность получившейся матрицы tfidf (635489, 1600000) до размерности (635489, 300) , используя SVD разложение

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

In [ ]:
# размер необходимого вектора ембеддинга
emb_size = word2vec.vectors.shape[1]
emb_size

300

In [ ]:
%%time

X_csr = csr_matrix(X.T)
svd = TruncatedSVD(n_components=emb_size, n_iter=5, random_state=42)
svd.fit(X_csr)

X_new = svd.transform(X_csr)

Wall time: 7min 19s


In [ ]:
# размерность ембеддинга
X_new.shape

(272542, 300)

In [ ]:
import pickle
with open('tfidf_embedding.pkl', 'wb') as tfidf_embedding_file:
    pickle.dump(X_new, tfidf_embedding_file)    
# with open('word2vec.pkl', 'rb') as word2vec_file:
#     word2vec = pickle.load(word2vec.pkl) 

Нормализуем матрицу

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
X_normilize = normalize(X_new, axis=1)

Собираем ембеддинги в словарь.

In [ ]:
tfidf_vec = {item:X_normilize[i] for i,item in enumerate(feature_list)}  

#### Модель word2vec + tfidf

Разработаем новый класс для загрузчика. Использовать будем сумму эмбеддингов word2vec и tfidf.

In [ ]:
class TwitterDataset_context_window_tfidf(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                tfidf_vec: dict):
        
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tfidf_vec = tfidf_vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)      
    
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"http?://[^,\s]+,?","",text)
        text = re.sub(r'@[\w]+',"",text)
        line = tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        filtered_line = [word for word in filtered_line if word not in stopWords]
        return filtered_line

    def get_embeddings_(self, tokens):
      
        embeddings = []       

        for idx, word in enumerate(tokens): 
            word_embedding =  self.word2vec.get_vector('python') *0.0
            find_emb = False
            if  word in self.word2vec:
                word_embedding += (self.word2vec.get_vector(word) - self.mean)/self.std
                find_emb = True
            if  word in self.tfidf_vec:
                word_embedding += self.tfidf_vec[word]
                find_emb = True
            if find_emb:
                embeddings.append(word_embedding)  
       
        # было ранее
        #embeddings = [((self.word2vec.get_vector(w) - self.mean)/self.std) for w in tokens if w in self.word2vec] 
  
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
 
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev_word2vec_tfidf = TwitterDataset_context_window_tfidf(dev_data, "text", "emotion", word2vec, tfidf_vec)

In [ ]:
# проверяем работоспособность
next(iter(dev_word2vec_tfidf))

{'feature': array([[-0.02615493, -1.2498533 ,  0.30613846, ...,  0.45147184,
          2.8304925 , -2.5080595 ],
        [-1.7594566 ,  0.02923906, -0.68109316, ...,  0.08646335,
         -1.132029  ,  0.69043213],
        [ 0.6142764 ,  0.78092414, -2.503302  , ...,  0.26314852,
          0.80110484,  0.39050657],
        ...,
        [ 0.9292662 ,  1.2742614 , -0.07795607, ..., -0.979371  ,
          0.74874896, -1.8027185 ],
        [ 0.3034666 ,  1.1909688 ,  1.9497143 , ..., -0.5160077 ,
          0.38224748, -0.7147519 ],
        [ 0.73338914,  0.16750537, -0.20423055, ..., -0.19839445,
          0.95621556, -0.53127974]], dtype=float32),
 'target': 0}

In [ ]:
batch_size = 1024
num_workers = 0 # 4 - для collab, 0 - local

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]
    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev_word2vec_tfidf) * 0.8)

train, valid = random_split(dev_word2vec_tfidf, [train_size, len(dev_word2vec_tfidf) - train_size])

#для local num_workers = 0
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, \
                          drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, \
                          drop_last=False, collate_fn=average_emb)

In [ ]:
model = MyClassifier(num_classes, vector_size) 
criterion = torch.nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters()) # Твой оптимайзер

In [ ]:
%%time

num_epochs = 5

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "8u.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5215481994152069, 'Test Acc': 0.7376484375}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5089289728403091, 'Test Acc': 0.7457578125}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5054598613977432, 'Test Acc': 0.74891015625}
Wall time: 23min 46s


In [ ]:
test_loader = DataLoader(
    TwitterDataset_context_window_tfidf(test_data, "text", "emotion", word2vec, tfidf_vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_word2vec_tfidf.pt", map_location=device))

res = testing(model, criterion, test_loader, device=device)
res['model'] = 'word2vec_tfidf' 
result_list.append(res)

print(res)

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5050846653434035, 'Test Acc': 0.7495600788738019, 'model': 'word2vec_tfidf'}


#### Модель tfidf

Разработаем новый класс для загрузчика. Использовать будем эмбеддинги только от tfidf.

In [ ]:
class TwitterDataset_tfidf(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, tfidf_vec: dict):
        
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.tfidf_vec = tfidf_vec

        self.label2num = lambda label: 0 if label == 0 else 1
    
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"http?://[^,\s]+,?","",text)
        text = re.sub(r'@[\w]+',"",text)
        line = tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        filtered_line = [word for word in filtered_line if word not in stopWords]
        return filtered_line

    def get_embeddings_(self, tokens):     
     
        embeddings = [self.tfidf_vec[w] for w in tokens if w in self.tfidf_vec] 
  
        if len(embeddings) == 0:
            embeddings = np.zeros((1, len(self.tfidf_vec['python'])))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
 
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev_tfidf = TwitterDataset_tfidf(dev_data, "text", "emotion", tfidf_vec)

In [ ]:
# проверим работоспособность
next(iter(dev_tfidf))

{'feature': array([[ 0.34376383, -0.13543272, -0.03109602, ...,  0.09354725,
          0.03781341,  0.1162918 ],
        [ 0.28541137, -0.12134647, -0.00409091, ...,  0.03636526,
         -0.05584035,  0.02064438],
        [ 0.39056758, -0.10716692, -0.02006971, ...,  0.07460458,
         -0.01679963,  0.07828123],
        [ 0.03587312, -0.02140533, -0.00550975, ..., -0.01044692,
         -0.01048095,  0.00174522],
        [ 0.28541137, -0.12134647, -0.00409091, ...,  0.03636526,
         -0.05584035,  0.02064438],
        [ 0.16970135, -0.00366991,  0.02041105, ...,  0.00086289,
         -0.00151392, -0.00183806]]),
 'target': 0}

In [ ]:
batch_size = 1024
num_workers = 0 # 4 - для collab, 0 - local

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]
    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev_tfidf) * 0.8)

train, valid = random_split(dev_tfidf, [train_size, len(dev_tfidf) - train_size])

#для local num_workers = 0
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, \
                          drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, \
                          drop_last=False, collate_fn=average_emb)

In [ ]:
model = MyClassifier(num_classes, vector_size) 
criterion = torch.nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters()) # Твой оптимайзер

In [ ]:
%%time

num_epochs = 5

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5231797255277634, 'Test Acc': 0.7365}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5111263029575348, 'Test Acc': 0.74498828125}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5097995635271072, 'Test Acc': 0.74565625}
Wall time: 14min 42s


In [ ]:
test_loader = DataLoader(
    TwitterDataset_tfidf(test_data, "text", "emotion", tfidf_vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

res = testing(model, criterion, test_loader, device=device)
res['model'] = 'tfidf' 
result_list.append(res)

print(res)

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5101874263141863, 'Test Acc': 0.7449923871805112, 'model': 'tfidf'}


Соберем все полученные результаты в итоговую табличку.

In [ ]:
pd_result = pd.DataFrame(result_list).sort_values(by=['Test Acc'], ascending=False)

# Итоги исследования

Были построены четыре различные модели для решения задачи семантической классификации твитов.

- **word2vec** - модель, в которой использовались ембеддинги предообученной модели от gensim. Для неищзвестных слов эмбеддинги пропускались.
- **word2vec context window** - модель, в которой ембеддинги неизвестных слов представлялись суммой ембеддингов контекста. Ширина контекста бралась в два токена слева и справа.
- **tfidf** - модель, для которой были получены ембеддинги благодаря **TfidfVectorizer** из sklearn. До требуемой размерности (300), матрица сжималась через **SVD** разложение.
- **word2vec_tfidf** - модель, в которой ембеддинг слова представлялся суммой ембеддингов от word2vec и tfidf.

Для решения задачи классификации (для двух классов) была построена простейшая нейросеть из трех полносвязных слоев, c батчнормализацией.

В качестве функции потерь при обучении нейросети использовалась кросс-энтропия. Для оценки качества модели была выбрана метрика accuracy.

Твиты в данных были частично предобработаны. Убраны стоп - слова, знаки пунктуации. Вырезаны http адреса и никнаймы @блабла.

Полученные результаты сведены в итоговую таблицу:

In [ ]:
pd_result

,Test Loss,Test Acc,model
2,0.505085,0.749560,word2vec_tfidf
0,0.508335,0.747679,word2vec
1,0.511253,0.745470,word2vec context window
3,0.510187,0.744992,tfidf


Наилучшие результаты показала модель word2vec_tfidf. По-видимому, за счет того, что в "контекстную" предобученную на "чужеродном" корпусе слов модель word2vec, была внесена добавочная информация о нашем конкретном корпусе твитов. Так как, tfidf обучался непосредственно на данных нашей задачи.